## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [1]:
# Only needed for Udacity workspace
import importlib.util
import sys

if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
# Import necessary libraries
import os
import json
import chromadb
from dotenv import load_dotenv
from tavily import TavilyClient
from pydantic import BaseModel, Field

# Custom library imports
from lib.agents import Agent
from lib.llm import LLM
from lib.tooling import tool
from chromadb.utils import embedding_functions

In [3]:
# Load environment variables
# load_dotenv()

OPENAI_API_KEY = "voc-11905559501688654276185689a2c33e68215.70173222"
TAVILY_API_KEY = "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"
VOCAREUM_API_BASE = "https://openai.vocareum.com/v1"

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [4]:
#### 1. Retrieve Game Tool
@tool # type: ignore
def retrieve_game(query: str) -> list:
    """Semantic search: Finds relevant results in the vector DB."""
    embedding_fn = embedding_functions.OpenAIEmbeddingFunction( # type: ignore
        api_key=OPENAI_API_KEY, # type: ignore
        api_base=VOCAREUM_API_BASE, # type: ignore
        model_name="text-embedding-ada-002"
    )
    chroma_client = chromadb.PersistentClient(path="chromadb") # type: ignore
    collection = chroma_client.get_collection(name="udaplay", embedding_function=embedding_fn)
    results = collection.query(query_texts=[query], n_results=5)
    return results['documents'][0] if results['documents'] else []

#### Evaluate Retrieval Tool

In [5]:
#### 2. Evaluate Retrieval Tool
class EvaluationReport(BaseModel): # type: ignore
    """Data model for the evaluation report."""
    useful: bool = Field(..., description="Whether the documents are useful to answer the question") # type: ignore
    description: str = Field(..., description="A detailed explanation of the evaluation result") # type: ignore

@tool # type: ignore
def evaluate_retrieval(question: str, retrieved_docs: list) -> EvaluationReport:
    """Analyzes the usability of retrieved documents to answer a question."""
    llm = LLM(model="gpt-4-turbo", temperature=0, api_key=OPENAI_API_KEY, base_url=VOCAREUM_API_BASE) # type: ignore
    docs_str = "\n\n".join(retrieved_docs)
    prompt = f"""Your task is to evaluate if the documents are enough to respond to the query.
    Respond with ONLY a JSON object in the following format:
    {{"useful": <true or false>, "description": "<Your detailed explanation here>"}}
    Original user question: {question}
    Retrieved documents: {docs_str}"""
    ai_message = llm.invoke(prompt)
    report_dict = json.loads(ai_message.content) # type: ignore
    return EvaluationReport(**report_dict)

#### Game Web Search Tool

In [6]:
@tool
def game_web_search(query: str) -> str:
    """Performs a web search using Tavily."""
    tavily_client = TavilyClient(api_key=TAVILY_API_KEY) # type: ignore
    response = tavily_client.search(query=query, search_depth="advanced")
    return "\n".join([result['content'] for result in response['results']])

### Agent

In [7]:
### Agent
instructions = """You are UdaPlay, an AI Research Agent for the video game industry.
Your workflow:
1. Use `retrieve_game` to search the internal database.
2. Use `evaluate_retrieval` to assess if the results are sufficient.
3. If not sufficient, use `game_web_search`.
4. Respond clearly and **you must cite your source**.
   - If from internal knowledge, end with `(Source: Internal Knowledge Base)`.
   - If from the web, end with `(Source: Web Search)`.
"""

udaplay_agent = Agent( # type: ignore
    model_name="gpt-4o-mini",
    instructions=instructions,
    tools=[retrieve_game, evaluate_retrieval, game_web_search], # type: ignore
    base_url=VOCAREUM_API_BASE, # type: ignore
    api_key=OPENAI_API_KEY # type: ignore
)

In [8]:
def run_and_print(query: str, session_id: str = None):
    """Helper function to run the agent and print the results clearly."""
    print(f"--- User Query: '{query}' ---")
    try:
        # Pass session_id ONLY for conversational queries
        run = udaplay_agent.invoke(query, session_id=session_id)
        final_state = run.get_final_state()
        
        print("\nAgent's Reasoning and Tool Usage:")
        if final_state and final_state.get('messages'):
            for message in final_state['messages']:
                if message.role == 'assistant' and message.tool_calls:
                    for tc in message.tool_calls:
                        print(f"- Tool Called: {tc.function.name} with args: {tc.function.arguments}")
            
            print(f"\nFinal Answer:\n{final_state['messages'][-1].content}\n")
        else:
            print("Agent run did not produce a final answer.")
            
    except Exception as e:
        print(f"An error occurred: {e}")
    print("-" * 40)

In [9]:
#### Section 1: Independent (Stateless) Queries
# These queries are run without a session_id, forcing the agent to start fresh each time.
# This demonstrates the fix for the state contamination issue.
print("### Demonstrating Independent (Stateless) Queries ###\n")
stateless_queries = [
    "When was Pokémon Gold and Silver released?",
    "Which one was the first 3D platformer Mario game?",
    "Was Mortal Kombat X released for PlayStation 5?"
]
for q in stateless_queries:
    run_and_print(q) # No session_id is passed

### Demonstrating Independent (Stateless) Queries ###

--- User Query: 'When was Pokémon Gold and Silver released?' ---
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Agent's Reasoning and Tool Usage:
- Tool Called: retrieve_game with args: {"query":"Pokémon Gold and Silver release date"}
- Tool Called: evaluate_retrieval with args: {"question":"When was Pokémon Gold and Silver released?","retrieved_docs":"Name: Pokémon Gold and Silver\nPlatform: Game Boy Color\nYear of Release: N/A\nGenre: Role-playing\nPublisher: Nintendo\nDescription: Second-generation Pokémon games introducing new regions, Pokémon

In [ ]:
#### Section 2: Conversational (Stateful) Queries
# These queries use the same session_id to show the agent can remember context.
print("\n\n### Demonstrating Conversational (Stateful) Queries ###\n")
conversation_session_id = "mario_chat_101"
conversational_queries = [
    "Tell me about 3D platformer Mario games in your database.",
    "Which one of those was released first?",
    "What platform was it for?"
]
for q in conversational_queries:
    run_and_print(q, session_id=conversation_session_id) # The SAME session_id is passed



--- User Query: Tell me about Mario games ---

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
Agent's Reasoning and Tool Usage:
- Tool Called: evaluate_retrieval
  - Arguments: {"question":"Tell me about Mario games","retrieved_docs":"Name: Super Mario 64\nPlatform: Nintendo 64\nYear of Release: N/A\nGenre: Platformer\nPublisher: Nintendo\nDescription: A groundbreaking 3D platformer that set new standards for the genre, featuring Mario's quest to rescue Princess Peach.\n\nName: Super Mario World\nPlatform: Super Nintendo Entertainment System (SNES)\nYear of Release: N/A\nGenre: Platformer\nPublisher: Nintendo\nDescription: A classic platformer where Mario embarks on a quest to save Princes